In [1]:
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [ ]:
# even the variables are not initialized yet
x =tf.Variable(initial_value=2, name='EE')

f = x**2+ 4*x - 5

init = tf.global_variables_initializer()  # prepare an init node

# the with clouse just make the session default
with tf.Session() as sess_01:
    # ses.run(x.initializer)
    # x.initializer.run() 
    
    # Instead of manually running the initializer for every single variable, you can use 
    # the global_variables_initializer() function.
    init.run()  # actually initialize all the variables
    
    # result = ses.run(f)
    result = f.eval()
    print('hi', result)

Inside Jupyter or within a Python shell you may prefer to create an InteractiveSession. The only difference from a regular Session is that when an InteractiveSession is created it automatically sets itself as the default session, so you don’t need a with block (but you do need to close the session manually when you are done with it)

In [ ]:
sess_02 = tf.Session()
sess_02.run(x.initializer)
result = sess_02.run(f)
print(result)

sess_02.close()

VS.

In [ ]:
sess_03 = tf.InteractiveSession()
init = tf.global_variables_initializer()  # prepare an init node
init.run()
result = f.eval()
print(result)

sess_03.close()

VS.

In [ ]:
sess_04 = tf.Session()
init = tf.global_variables_initializer()  # prepare an init node

init.run(session=sess_04)
result = f.eval(session=sess_04)
print(result)

sess_04.close()

In [ ]:
# Any node you create is automatically added to the default graph:
x1 = tf.Variable(1)
print(x1.graph)
x1.graph is tf.get_default_graph()

graph_01 = tf.Graph()
with graph_01.as_default():
    x2 = tf.Variable(2)

print(x2.graph is graph_01)

# graph_01 is only default within the 'with' scope
print(x2.graph is tf.get_default_graph())

Preparing the Data 
===

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

scaler = StandardScaler()
scaler.fit(housing_data_plus_bias)
# print(scaler.mean_)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)
trg = housing.target.reshape(-1, 1)
trg.shape

(20640, 1)

Linear Regression with TensorFlow via Normal Equation
===

In [ ]:
# Constants and variables take no input (they are called source ops).
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

# the Normal Equation
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:    
    heta_value = theta.eval()
    

Linear Regression with TensorFlow via Optimizer
===

In [5]:
n_epochs = 1000
learning_rate = 0.01

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    # [...] # load the data from disk
    Xf = scaled_housing_data_plus_bias[batch_index:batch_index+batch_size,:]
    yf = trg[batch_index:batch_index+batch_size,:]
    
#     X_batch = tf.constant(Xf, dtype=tf.float32, name="X")
#     y_batch = tf.constant(yf, dtype=tf.float32, name="y")
#     print(X_batch.dtype, X_batch.shape)
    return Xf, yf

# X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
# y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# placeholder nodes don’t actually perform any computation, 
# they just output the data you tell them to output at runtime.
# If you specify None for a dimension, it means “any size.”
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")



theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
# the hypothesis
y_pred = tf.matmul(X, theta, name="predictions")

# The cost function; Mean Square Average
error = y_pred - y
cost_func = tf.reduce_mean(tf.square(error), name="mse")

# gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(cost_func, [theta])[0]

# training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(cost_func)

init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
#             if epoch % 100 == 0:
#                 print("Epoch", epoch, "MSE =", cost_func.eval())
            # sess.run(training_op)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        

    best_theta = theta.eval()
    save_path = saver.save(sess, "/home/disooqi/my_model_final.ckpt")